In [129]:
import json
import urllib3
import config
import pandas as pd
import string
import datetime

http = urllib3.PoolManager()

# http://api.openweathermap.org/geo/1.0/direct?q={city name},{state code},{country code}&limit={limit}&appid={API key}
location = [
    'New York City, NY, USA',
    'Los Angeles, CA, USA',
    'Chicago, IL, USA',
    'Houston, TX, USA',
    'Phoenix, AZ, USA',
    'Philadelphia, PA, USA',
    'San Antonio, TX, USA',
    'San Diego, CA, USA',
    'Dallas, TX, USA',
    'San Jose, CA, USA',
    'Austin, TX, USA',
    'Jacksonville, FL, USA',
    'Fort Worth, TX, USA',
    'Columbus, OH, USA',
    'San Francisco, CA, USA',
    'Charlotte, NC, USA',
    'Indianapolis, IN, USA',
    'Seattle, WA, USA',
    'Denver, CO, USA',
    'Washington, D.C., DC, USA',    
]

# append to empty df doesn't happen in place; need to store it first
# https://stackoverflow.com/questions/16597265/appending-to-an-empty-dataframe-in-pandas
loc_responses = []
all_weather_data = []
weather_data = []

for l in location:
    # Location call
    call_string = 'https://api.openweathermap.org/geo/1.0/direct?q=' + l + '&limit=1&appid=' + config.api_key
    loc_response = json.loads(http.request('GET', call_string).data)
    loc_responses.append(loc_response)
    #locations_df = locations_df.append(pd.json_normalize(loc_response))
    print(l, loc_response[0]['lat'], loc_response[0]['lon'])
    # Weather call
    call_string = 'https://api.openweathermap.org/data/2.5/weather?lat=' + str(loc_response[0]['lat']) + '&lon=' + str(loc_response[0]['lon']) + '&appid=' + config.api_key
    weather_response = json.loads(http.request('GET', call_string).data)
    all_weather_data.append(weather_response)
    # useful for processing json: https://stackoverflow.com/questions/67745643/select-specific-keys-inside-a-json-using-python
    weather_trim = { 
        'city' : weather_response['name'],
        'description' : weather_response['weather'][0]['description'],
        'icon': weather_response['weather'][0]['icon'],   
        'current_temp': weather_response['main']['temp'] - 273.15,
        'feels_like': weather_response['main']['feels_like'] - 273.15,
        'temp_min': weather_response['main']['temp_min'] - 273.15,
        'temp_max': weather_response['main']['temp_max'] - 273.15,
        'visibility': weather_response['visibility'],
        'sunrise': datetime.datetime.fromtimestamp(weather_response['sys']['sunrise']),
        'sunset': datetime.datetime.fromtimestamp(weather_response['sys']['sunset']),
        'current_time':  datetime.datetime.fromtimestamp(weather_response['dt']),
        'timezone': weather_response['timezone']
    }
    weather_data.append(weather_trim)

New York City, NY, USA 40.7127281 -74.0060152
Los Angeles, CA, USA 34.0536909 -118.242766
Chicago, IL, USA 41.8755616 -87.6244212
Houston, TX, USA 29.7589382 -95.3676974
Phoenix, AZ, USA 33.4484367 -112.074141
Philadelphia, PA, USA 39.9527237 -75.1635262
San Antonio, TX, USA 29.4246002 -98.4951405
San Diego, CA, USA 32.7174202 -117.1627728
Dallas, TX, USA 32.7762719 -96.7968559
San Jose, CA, USA 37.3361663 -121.890591
Austin, TX, USA 30.2711286 -97.7436995
Jacksonville, FL, USA 30.3321838 -81.655651
Fort Worth, TX, USA 32.753177 -97.3327459
Columbus, OH, USA 39.9622601 -83.0007065
San Francisco, CA, USA 37.7790262 -122.419906
Charlotte, NC, USA 35.2272086 -80.8430827
Indianapolis, IN, USA 39.7683331 -86.1583502
Seattle, WA, USA 47.6038321 -122.330062
Denver, CO, USA 39.7392364 -104.984862
Washington, D.C., DC, USA 38.895 -77.0366


In [130]:
weather_df = pd.json_normalize(weather_data)
weather_df.head()

,city,description,icon,current_temp,feels_like,temp_min,temp_max,visibility,sunrise,sunset,current_time,timezone
0,New York,clear sky,01d,24.75,24.51,23.15,25.61,10000,2024-07-01 02:28:42,2024-07-01 17:30:57,2024-07-01 17:23:25,-14400
1,Los Angeles,clear sky,01d,27.00,27.67,21.80,31.11,10000,2024-07-01 05:45:22,2024-07-01 20:08:14,2024-07-01 17:35:02,-25200
2,Chicago,broken clouds,04d,20.84,20.57,19.47,22.46,10000,2024-07-01 03:19:16,2024-07-01 18:29:20,2024-07-01 17:30:54,-18000
3,Houston,scattered clouds,03d,34.61,41.61,31.64,36.29,10000,2024-07-01 04:24:38,2024-07-01 18:25:56,2024-07-01 17:29:46,-18000
4,Phoenix,broken clouds,04d,39.50,42.17,38.06,41.10,10000,2024-07-01 05:22:17,2024-07-01 19:41:58,2024-07-01 17:24:12,-25200


In [131]:
# Weather forecast
# api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={API key}
all_forecasts = []
forecast_data = []

for l in loc_responses:
    call_string = 'https://api.openweathermap.org/data/2.5/forecast?lat=' + str(l[0]['lat']) + '&lon=' + str(l[0]['lon']) + '&appid=' + config.api_key
    forecast_response = json.loads(http.request('GET', call_string).data)
    all_forecasts.append(forecast_response)
    for l in forecast_response['list']:
        forecast_trim = { 
            'city' : forecast_response['city']['name'],
            'description' : l['weather'][0]['description'],
            'icon': l['weather'][0]['icon'],   
            'current_temp': l['main']['temp'] - 273.15,
            'feels_like': l['main']['feels_like'] - 273.15,
            'temp_min': l['main']['temp_min'] - 273.15,
            'temp_max': l['main']['temp_max'] - 273.15,
            'visibility': l['visibility'],
            # 'sunrise': datetime.datetime.fromtimestamp(forecast_response['city']['sunrise']),
            # 'sunset': datetime.datetime.fromtimestamp(forecast_response['city']['sunset']),
            'forecast_time':  datetime.datetime.fromtimestamp(l['dt']),
            'timezone': forecast_response['city']['timezone']
        }
        forecast_data.append(forecast_trim)

In [132]:
forecast_df = pd.json_normalize(forecast_data)
forecast_df.head(100)

,city,description,icon,current_temp,feels_like,temp_min,temp_max,visibility,forecast_time,timezone
0,New York,clear sky,01n,24.25,23.91,23.20,24.25,10000,2024-07-01 20:00:00,-14400
1,New York,clear sky,01n,22.27,21.75,21.02,22.27,10000,2024-07-01 23:00:00,-14400
2,New York,clear sky,01n,19.28,18.60,19.28,19.28,10000,2024-07-02 02:00:00,-14400
3,New York,clear sky,01d,20.68,20.03,20.68,20.68,10000,2024-07-02 05:00:00,-14400
4,New York,clear sky,01d,25.71,25.25,25.71,25.71,10000,2024-07-02 08:00:00,-14400
...,...,...,...,...,...,...,...,...,...,...
95,Chicago,broken clouds,04d,26.29,26.29,26.29,26.29,10000,2024-07-03 17:00:00,-18000
96,Chicago,broken clouds,04n,24.08,24.32,24.08,24.08,10000,2024-07-03 20:00:00,-18000
97,Chicago,broken clouds,04n,22.71,23.10,22.71,22.71,10000,2024-07-03 23:00:00,-18000
98,Chicago,overcast clouds,04n,21.74,22.14,21.74,21.74,10000,2024-07-04 02:00:00,-18000
